# Mono resistance prediction

In this POC, we gloss over the resistance towards distinct drugs


In [1]:
# Import the usual suspects.

from __future__ import print_function
import pandas as pd
import numpy as np

In [2]:
lineage_column_names = [ 'main_lin', 'sublin' ]

resistance_status_column_names = [ 'drtype', 'MDR', 'XDR', 'isResistant' ]


In [3]:
renamed_drug_columns_names = [                     'rifampicin_resistance',
                                                   'isoniazid_resistance',
                                                   'pyrazinamide_resistance',
                                                   'ethambutol_resistance',
                                                   'streptomycin_resistance',
                                                   'fluoroquinolones_resistance',
                                                   'moxifloxacin_resistance',
                                                   'ofloxacin_resistance',
                                                   'levofloxacin_resistance',
                                                   'ciprofloxacin_resistance',
                                                   'aminoglycosides_resistance',
                                                   'amikacin_resistance',
                                                   'kanamycin_resistance',
                                                   'capreomycin_resistance',
                                                   'ethionamide_resistance',
                                                   'para-aminosalicylic_acid_resistance',
                                                   'cycloserine_resistance',
                                                   'linezolid_resistance',
                                                   'bedaquiline_resistance',
                                                   'clofazimine_resistance',
                                                   'delamanid_resistance']


In [ ]:

binary_unique_snps_df = pd.read_csv("../data/processed/binary_unique_snps_df.csv")
binary_unique_snps_df.head()

In [ ]:
# Ensure that every column has atleast one mutation

binary_col_mutation_dict = {}

for col in binary_unique_snps_df.columns:
    sum_of_col = binary_unique_snps_df[col].sum()
    if  sum_of_col > 0:
        # print('col: ', col, "\tsum: ", col_sum)
        binary_col_mutation_dict[col] =  dict(binary_unique_snps_df[col].value_counts())

binary_col_mutation_dict


In [ ]:
binary_unique_snps_df = binary_unique_snps_df.rename({'Unnamed: 0': 'SampleID'}).set_index('SampleID')
binary_unique_snps_df.head()


In [ ]:
# hetero_binary_vcf_snps_with_mutations_df = pd.read_csv("../data/processed/hetero_binary_vcf_snps_with_mutations_df.csv", index_col='SampleID')

hetero_binary_vcf_snps_with_mutations_df = binary_unique_snps_df[list(binary_col_mutation_dict.keys())]

hetero_binary_vcf_snps_with_mutations_df.head()


In [ ]:
resistance_status_df = pd.read_csv("../data/processed/resistance_status_df.csv").rename(columns={'Unnamed: 0' : 'SampleID'}).set_index('SampleID')

resistance_status_df.head()


In [ ]:
final_df = hetero_binary_vcf_snps_with_mutations_df.join(resistance_status_df)
# cols = list(final_df.columns)
# cols = [*cols[1:], cols[0]]
# final_df = final_df[cols]
final_df.head()

In [ ]:

# final_df['bedaquiline_resistance'].value_counts()

In [ ]:
final_df.to_csv("../data/processed/final_df.csv")

In [ ]:
final_df = pd.read_csv("../data/processed/final_df.csv")
# final_df = pd.read_json("../data/processed/final_df.json")

final_df.head()

In [ ]:

final_df.rename(columns = {'Unnamed: 0': 'SampleID'},
                            inplace=True)

final_df.head()

In [ ]:
final_df = final_df.set_index('SampleID')

final_df.head()


In [10]:
mono_resistance_df = final_df.drop(columns=[*lineage_column_names, *renamed_drug_columns_names, 'MDR', 'XDR', 'drtype'])
mono_resistance_df.head()


,NC000962_3.11,NC000962_3.78,NC000962_3.80,NC000962_3.82,NC000962_3.88,NC000962_3.102,NC000962_3.104,NC000962_3.108,NC000962_3.117,NC000962_3.120,...,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR3129939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
ERR3148148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
ERR3148149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
ERR3148151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
ERR3148153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [12]:
mono_resistance_df_small = mono_resistance_df.iloc[:, -10:]
mono_resistance_df_small.to_csv("../data/processed/mono_resistance_df_small.csv")
mono_resistance_df_small.head()


,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,
ERR3129939,0,0,0,0,0,0,0,0,0,0.0
ERR3148148,0,0,0,0,0,0,0,0,0,0.0
ERR3148149,0,0,0,0,0,0,0,0,0,1.0
ERR3148151,0,0,0,0,0,0,0,0,0,1.0
ERR3148153,0,0,0,0,0,0,0,0,0,0.0


In [11]:
mono_resistance_df.to_csv("../data/processed/mono_resistance_df.csv")
mono_resistance_df.head()


,NC000962_3.11,NC000962_3.78,NC000962_3.80,NC000962_3.82,NC000962_3.88,NC000962_3.102,NC000962_3.104,NC000962_3.108,NC000962_3.117,NC000962_3.120,...,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR3129939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
ERR3148148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
ERR3148149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
ERR3148151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
ERR3148153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [37]:
mono_resistance_df

,NC000962_3.11,NC000962_3.78,NC000962_3.80,NC000962_3.82,NC000962_3.88,NC000962_3.102,NC000962_3.104,NC000962_3.108,NC000962_3.117,NC000962_3.120,...,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR3129939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
ERR3148148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
ERR3148149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
ERR3148151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
ERR3148153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR9224981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
SRR9224985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
SRR9224986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


# Scikit-learn


In [38]:
mono_resistance_df['NC000962_3.11'].unique()

array([0], dtype=int64)

In [34]:
non_zero_mono_resistance_df = mono_resistance_df[mono_resistance_df > 0]
non_zero_mono_resistance_df

,NC000962_3.11,NC000962_3.78,NC000962_3.80,NC000962_3.82,NC000962_3.88,NC000962_3.102,NC000962_3.104,NC000962_3.108,NC000962_3.117,NC000962_3.120,...,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR3129939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ERR3148148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ERR3148149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
ERR3148151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
ERR3148153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR9224981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
SRR9224985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
SRR9224986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [35]:
non_zero_mono_resistance_df = non_zero_mono_resistance_df.fillna(0)
non_zero_mono_resistance_df



,NC000962_3.11,NC000962_3.78,NC000962_3.80,NC000962_3.82,NC000962_3.88,NC000962_3.102,NC000962_3.104,NC000962_3.108,NC000962_3.117,NC000962_3.120,...,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR3129939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3148148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR3148149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ERR3148151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ERR3148153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR9224981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SRR9224985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
SRR9224986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [36]:
non_zero_mono_resistance_df.sum()



NC000962_3.11           0.0
NC000962_3.78           0.0
NC000962_3.80           0.0
NC000962_3.82           0.0
NC000962_3.88           0.0
                      ...  
NC000962_3.4411245      0.0
NC000962_3.4411286      0.0
NC000962_3.4411405      0.0
NC000962_3.4411518      0.0
isResistant           239.0
Length: 134257, dtype: float64

In [21]:
mono_resistance_df['ERR3129939']

KeyError: "None of [Index(['ERR3129939'], dtype='object')] are in the [columns]"

# H2O

In [13]:
# NOTE: here onwards this is the first iteration for the stacked ensemble

import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.estimators import H2OGeneralizedLowRankEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init(min_mem_size='12G')



Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 11.0.2-BellSoft+7, mixed mode)
  Starting server from C:\tools\miniconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\abhi1\AppData\Local\Temp\tmpaomo4u0a
  JVM stdout: C:\Users\abhi1\AppData\Local\Temp\tmpaomo4u0a\h2o_abhi1_started_from_python.out
  JVM stderr: C:\Users\abhi1\AppData\Local\Temp\tmpaomo4u0a\h2o_abhi1_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,3 months and 27 days !!!
H2O_cluster_name:,H2O_from_python_abhi1_m58c2e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
mono_resistance_df = pd.read_csv("../data/processed/mono_resistance_df.csv", index_col='SampleID')
mono_resistance_df.head()

,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,
ERR3129939,0,0,0,0,0,0,0,0,0,0.0
ERR3148148,0,0,0,0,0,0,0,0,0,0.0
ERR3148149,0,0,0,0,0,0,0,0,0,1.0
ERR3148151,0,0,0,0,0,0,0,0,0,1.0
ERR3148153,0,0,0,0,0,0,0,0,0,0.0


In [36]:
# SMALL
mono_resistance_df = pd.read_csv("C:/Users/abhi1/Projects/code/drug-resistance-prediction-cambiohack/src/data/processed/mono_resistance_df_small.csv", index_col='SampleID')
mono_resistance_df.head()

,NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
SampleID,,,,,,,,,,
ERR3129939,0,0,0,0,0,0,0,0,0,0.0
ERR3148148,0,0,0,0,0,0,0,0,0,0.0
ERR3148149,0,0,0,0,0,0,0,0,0,1.0
ERR3148151,0,0,0,0,0,0,0,0,0,1.0
ERR3148153,0,0,0,0,0,0,0,0,0,0.0


In [47]:
mono_resistance_frame = h2o.H2OFrame(mono_resistance_df)
mono_resistance_frame.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


NC000962_3.11,NC000962_3.78,NC000962_3.80,NC000962_3.82,NC000962_3.88,NC000962_3.102,NC000962_3.104,NC000962_3.108,NC000962_3.117,NC000962_3.120,NC000962_3.135,NC000962_3.138,NC000962_3.150,NC000962_3.155,NC000962_3.156,NC000962_3.159,NC000962_3.160,NC000962_3.187,NC000962_3.189,NC000962_3.192,NC000962_3.204,NC000962_3.207,NC000962_3.210,NC000962_3.213,NC000962_3.219,NC000962_3.225,NC000962_3.228,NC000962_3.231,NC000962_3.237,NC000962_3.246,NC000962_3.255,NC000962_3.267,NC000962_3.270,NC000962_3.273,NC000962_3.333,NC000962_3.371,NC000962_3.467,NC000962_3.571,NC000962_3.576,NC000962_3.582,NC000962_3.591,NC000962_3.600,NC000962_3.609,NC000962_3.610,NC000962_3.636,NC000962_3.645,NC000962_3.654,NC000962_3.660,NC000962_3.663,NC000962_3.669,NC000962_3.672,NC000962_3.675,NC000962_3.678,NC000962_3.682,NC000962_3.684,NC000962_3.690,NC000962_3.693,NC000962_3.705,NC000962_3.708,NC000962_3.741,NC000962_3.750,NC000962_3.756,NC000962_3.771,NC000962_3.777,NC000962_3.786,NC000962_3.795,NC000962_3.805,NC000962_3.813,NC000962_3.822,NC000962_3.828,NC000962_3.831,NC000962_3.837,NC000962_3.840,NC000962_3.843,NC000962_3.846,NC000962_3.849,NC000962_3.876,NC000962_3.877,NC000962_3.885,NC000962_3.897,NC000962_3.903,NC000962_3.909,NC000962_3.912,NC000962_3.918,NC000962_3.921,NC000962_3.927,NC000962_3.933,NC000962_3.942,NC000962_3.945,NC000962_3.955,NC000962_3.963,NC000962_3.966,NC000962_3.975,NC000962_3.984,NC000962_3.990,NC000962_3.993,NC000962_3.996,NC000962_3.1002,NC000962_3.1005,NC000962_3.1008,NC000962_3.1020,NC000962_3.1024,NC000962_3.1035,NC000962_3.1047,NC000962_3.1050,NC000962_3.1053,NC000962_3.1059,NC000962_3.1068,NC000962_3.1071,NC000962_3.1074,NC000962_3.1092,NC000962_3.1098,NC000962_3.1104,NC000962_3.1110,NC000962_3.1125,NC000962_3.1152,NC000962_3.1153,NC000962_3.1161,NC000962_3.1164,NC000962_3.1167,NC000962_3.1176,NC000962_3.1177,NC000962_3.1191,NC000962_3.1197,NC000962_3.1204,NC000962_3.1220,NC000962_3.1230,NC000962_3.1242,NC000962_3.1245,NC000962_3.1254,NC000962_3.1278,NC000962_3.1287,NC000962_3.1290,NC000962_3.1293,NC000962_3.1364,NC000962_3.1365,NC000962_3.1371,NC000962_3.1375,NC000962_3.1383,NC000962_3.1389,NC000962_3.1392,NC000962_3.1395,NC000962_3.1404,NC000962_3.1407,NC000962_3.1413,NC000962_3.1416,NC000962_3.1419,NC000962_3.1422,NC000962_3.1431,NC000962_3.1456,NC000962_3.1461,NC000962_3.1464,NC000962_3.1470,NC000962_3.1473,NC000962_3.1478,NC000962_3.1485,NC000962_3.1488,NC000962_3.1503,NC000962_3.1590,NC000962_3.1662,NC000962_3.1701,NC000962_3.1763,NC000962_3.1821,NC000962_3.1849,NC000962_3.1928,NC000962_3.1977,NC000962_3.1993,NC000962_3.2092,NC000962_3.2097,NC000962_3.2103,NC000962_3.2105,NC000962_3.2108,NC000962_3.2123,NC000962_3.2141,NC000962_3.2144,NC000962_3.2146,NC000962_3.2147,NC000962_3.2153,NC000962_3.2159,NC000962_3.2183,NC000962_3.2187,NC000962_3.2190,NC000962_3.2193,NC000962_3.2199,NC000962_3.2201,NC000962_3.2205,NC000962_3.2207,NC000962_3.2210,NC000962_3.2213,NC000962_3.2216,NC000962_3.2219,NC000962_3.2249,NC000962_3.2264,NC000962_3.2270,NC000962_3.2272,NC000962_3.2279,NC000962_3.2285,NC000962_3.2291,NC000962_3.2294,NC000962_3.2297
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [8]:
mono_resistance_df.columns[-1]

'isResistant'

In [6]:
mono_resistance_df['isResistant'].head()

isResistant
0
0
1
1
0
0
1
1
1
1


In [16]:
train, test = mono_resistance_frame.split_frame(ratios=[.70], seed=1234)

In [17]:
test.head()


NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1


In [18]:
train.head()


NC000962_3.4410070,NC000962_3.4410276,NC000962_3.4410309,NC000962_3.4410847,NC000962_3.4411128,NC000962_3.4411245,NC000962_3.4411286,NC000962_3.4411405,NC000962_3.4411518,isResistant
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,1


In [ ]:
h2o.export_file(mono_resistance_frame, "../data/processed/mono_resistance_frame.csv")

Export File progress: |██

In [27]:
# Import a sample binary outcome train/test set into H2O
train_higgs = h2o.import_file("../data/external/higgs_train_10k.csv")
test_higgs = h2o.import_file("../data/external/higgs_test_5k.csv")


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
train = train_higgs
train_higgs.head()

x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,response
0.869,-0.635,0.226,0.327,-0.69,0.754,-0.249,-1.09,0,1.37,-0.654,0.93,1.11,1.14,-1.58,-1.05,0,0.658,-0.0105,-0.0458,3.1,1.35,0.98,0.978,0.92,0.722,0.989,0.877,1
0.908,0.329,0.359,1.5,-0.313,1.1,-0.558,-1.59,2.17,0.813,-0.214,1.27,2.21,0.5,-1.26,0.732,0,0.399,-1.14,-0.000819,0,0.302,0.833,0.986,0.978,0.78,0.992,0.798,1
0.799,1.47,-1.64,0.454,0.426,1.1,1.28,1.38,0,0.852,1.54,-0.82,2.21,0.993,0.356,-0.209,2.55,1.26,1.13,0.9,0,0.91,1.11,0.986,0.951,0.803,0.866,0.78,1
1.34,-0.877,0.936,1.99,0.882,1.79,-1.65,-0.942,0,2.42,-0.676,0.736,2.21,1.3,-1.43,-0.365,0,0.745,-0.678,-1.36,0,0.947,1.03,0.999,0.728,0.869,1.03,0.958,0
1.11,0.321,1.52,0.883,-1.21,0.681,-1.07,-0.922,0,0.801,1.02,0.971,2.21,0.597,-0.35,0.631,0,0.48,-0.374,0.113,0,0.756,1.36,0.987,0.838,1.13,0.872,0.808,1
1.6,-0.608,0.00707,1.82,-0.112,0.848,-0.566,1.58,2.17,0.755,0.643,1.43,0,0.922,-1.19,-1.62,0,0.651,-0.654,-1.27,3.1,0.824,0.938,0.972,0.789,0.431,0.961,0.958,0
0.409,-1.88,-1.03,1.67,-1.6,1.34,0.0554,0.0135,2.17,0.51,-1.04,0.708,0,0.747,-0.358,-1.65,0,0.367,0.0695,1.38,3.1,0.869,1.22,1,0.545,0.699,0.977,0.829,1
0.934,0.629,0.528,0.238,-0.967,0.548,-0.0594,-1.71,2.17,0.941,-2.65,-0.157,0,1.03,-0.176,0.523,2.55,1.37,1.29,-1.47,0,0.902,1.08,0.98,0.783,0.849,0.894,0.775,1
1.41,0.537,0.69,1.18,-0.11,3.2,-1.53,-1.58,0,2.93,0.567,-0.13,2.21,1.79,0.899,0.585,2.55,0.402,-0.151,1.16,0,1.67,4.04,1.18,1.05,1.54,3.53,2.74,1
1.18,0.104,1.4,0.48,0.266,1.14,1.53,-0.253,0,1.03,0.534,1.18,0,2.41,0.0876,-0.977,2.55,1.25,0.269,0.53,0,0.833,0.774,0.986,1.1,0.849,0.937,0.812,1


In [29]:
test = test_higgs
test_higgs.head()


x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,response
0.623878,0.459659,-0.316512,1.55701,0.644784,0.455929,-2.24983,-0.361949,0,0.832978,1.41147,1.52846,0,0.722857,1.00964,0.999539,0,1.19099,0.81404,-0.879803,3.10196,0.583344,0.790246,1.04087,0.848858,0.229702,0.65594,0.677857,0
1.68204,0.774251,1.32875,0.234729,-0.55783,0.442921,-0.0832048,1.08119,2.17308,1.14585,0.16228,-0.268187,0,1.21407,0.194966,-1.15627,2.54822,1.01915,-0.759163,0.136347,0,0.94183,0.941976,0.98816,0.86435,0.835132,0.767871,0.833013,0
1.09403,-0.88734,0.949229,0.410261,-1.55868,0.523352,0.131675,0.304513,0,1.11916,1.09383,-0.396352,2.21487,0.6289,0.628245,1.45775,0,0.523082,0.508394,0.988137,0,0.929989,0.973742,0.989141,0.780519,0.749821,0.887545,0.925278,1
1.33871,-0.899027,-1.25534,0.884285,-0.747925,0.547445,-0.79518,0.174235,2.17308,0.771663,-0.31855,1.02356,2.21487,0.408495,-1.1431,0.139699,0,1.29395,-1.34131,1.61685,0,0.79204,0.76921,0.984082,1.06928,0.700852,0.83083,0.71671,0
0.699095,1.43363,-1.72253,0.650113,-0.167574,0.967464,0.737696,-0.886391,2.17308,1.66797,0.446892,0.851008,0,0.59641,0.156736,0.340513,0,0.694684,-0.00046069,-1.14949,3.10196,0.714182,0.839843,0.984067,0.917766,0.388553,0.91664,0.886331,0
0.842208,-0.144202,-1.702,0.685875,0.544338,0.578774,-0.221837,-0.586474,1.08654,0.909401,2.81316,-0.0706666,0,1.54986,-0.85182,1.58811,0,1.14986,0.590011,0.0597691,0,0.872279,0.684341,0.99032,0.510151,0.720446,0.555454,0.535795,1
0.456242,1.74043,0.258427,1.45223,-0.0513728,0.675328,-0.797161,-1.34763,2.17308,0.56858,-0.482712,-1.73738,0,0.539333,1.17621,1.06666,0,0.980687,-0.125384,0.0869598,3.10196,0.994118,0.960552,1.00353,0.57609,0.876509,0.85138,0.801164,1
0.933712,1.35084,0.773338,0.532638,-0.679537,0.718017,0.745618,-1.4962,1.08654,0.274218,1.07537,1.56341,0,0.39146,-0.59422,0.170209,2.54822,0.573711,2.12157,0.256763,0,0.625972,0.773067,0.987348,0.674946,0.829309,0.645423,0.597715,0
0.7335,-0.998372,-1.68702,0.951169,-0.591779,1.14912,-0.942724,-0.424039,2.17308,1.04185,1.10063,1.27434,0,0.713549,0.0201984,0.343287,2.54822,1.44706,0.165271,1.40599,0,0.798081,0.852638,0.990337,0.799596,0.918739,1.27283,1.07795,0
0.968118,1.719,0.030934,1.80122,-0.867323,0.631357,-0.810034,1.20648,0,0.88699,0.00103165,1.57395,0,1.02721,0.512643,0.381009,2.54822,1.08511,0.54837,-0.86704,3.10196,0.844315,1.03113,1.32327,1.11639,0.728902,0.805447,1.19234,0


In [19]:
# Identify predictors and response
x = train.columns
y = "isResistant"
# y = "response"
x.remove(y)

In [20]:
x[:10]



['NC000962_3.4410070',
 'NC000962_3.4410276',
 'NC000962_3.4410309',
 'NC000962_3.4410847',
 'NC000962_3.4411128',
 'NC000962_3.4411245',
 'NC000962_3.4411286',
 'NC000962_3.4411405',
 'NC000962_3.4411518']

In [21]:
y


'isResistant'

In [22]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()


In [28]:
for x in train.columns:
    train[x] = train[x].asfactor()
    train[y] = train[y].asfactor()

In [29]:
train[y].head()

isResistant
0
1
0
0
1
1
1
1
1
1


In [30]:
test[y].head()

isResistant
0
1
1
1
1
1
1
1
0
1


In [25]:
# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

# There are a few ways to assemble a list of models to stack together:
# 1. Train individual models and put them in a list
# 2. Train a grid of models
# 3. Train several grids of models
# Note: All base models must have the same cross-validation folds and
# the cross-validated predicted values must be kept.


# 1. Generate a 2-model ensemble (GBM + RF)


In [31]:

# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(k = 2,
                                            loss = "quadratic",
                                            gamma_x = 0.5,
                                            gamma_y = 0.5,
                                            max_iterations = 700,
                                            recover_svd = True,
                                            init = "SVD",
                                            transform = "standardize")
glrm_model.train(training_frame=train)

H2OResponseError: ModelBuilderErrorV3  (water.exceptions.H2OModelBuilderIllegalArgumentException):
    timestamp = 1601345808209
    error_url = '/3/ModelBuilders/glrm'
    msg = 'Illegal argument(s) for GLRM model: GLRM_model_python_1601345407067_2.  Details: ERRR on field: _train: _train must have more than one column'
    dev_msg = 'Illegal argument(s) for GLRM model: GLRM_model_python_1601345407067_2.  Details: ERRR on field: _train: _train must have more than one column'
    http_status = 412
    values = {'messages': [{'_log_level': 5, '_field_name': '_keep_cross_validation_models', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_keep_cross_validation_predictions', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_keep_cross_validation_fold_assignment', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': '_fold_assignment', '_message': 'Only for cross-validation.'}, {'_log_level': 5, '_field_name': 

In [38]:

# Train and cross-validate a GBM
my_gbm = H2OGradientBoostingEstimator(distribution="bernoulli",
                                      ntrees=10,
                                      max_depth=3,
                                      min_rows=2,
                                      learn_rate=0.2,
                                      nfolds=nfolds,
                                      fold_assignment="Modulo",
                                      keep_cross_validation_predictions=True,
                                      seed=1)

my_gbm.train(x=x, y=y, training_frame=train)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [39]:

# Train and cross-validate a RF
my_rf = H2ORandomForestEstimator(ntrees=50,
                                 nfolds=nfolds,
                                 fold_assignment="Modulo",
                                 keep_cross_validation_predictions=True,
                                 seed=1)

my_rf.train(x=x, y=y, training_frame=train)


drf Model Build progress: |███████████████████████████████████████████████| 100%


In [40]:
# Train a stacked ensemble using the GBM and GLM above
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_binomial",
                                       base_models=[my_gbm, my_rf])
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
perf_gbm_test = my_gbm.model_performance(test)
perf_rf_test = my_rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)


# 2. Generate a random grid of models and stack them together

# Specify GBM hyperparameters for the grid
hyper_params = {"learn_rate": [0.01, 0.03],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}

# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees=10,
                                                        seed=1,
                                                        nfolds=nfolds,
                                                        fold_assignment="Modulo",
                                                        keep_cross_validation_predictions=True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
grid.train(x=x, y=y, training_frame=train)

# Train a stacked ensemble using the GBM grid
ensemble = H2OStackedEnsembleEstimator(model_id="my_ensemble_gbm_grid_binomial",
                                       base_models=grid.model_ids)
ensemble.train(x=x, y=y, training_frame=train)

# Eval ensemble performance on the test data
perf_stack_test = ensemble.model_performance(test)

# Compare to base learner performance on the test set
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

# Generate predictions on a test set (if neccessary)
pred = ensemble.predict(test)


stackedensemble Model Build progress: |███████████████████████████████████| 100%
Best Base-learner Test AUC:  0.7702913152422667
Ensemble Test AUC:  0.7745421930491371
stackedensemble prediction progress: |████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
stackedensemble Model Build progress: |███████████████████████████████████| 100%
Best Base-learner Test AUC:  0.7477238156143039
Ensemble Test AUC:  0.7490942802788068
stackedensemble prediction progress: |████████████████████████████████████| 100%
